<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/retirement_planning_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

numsims = 10000

# example data
initial_balance = 250000
years_saving = 30
years_withdrawing = 30
initial_saving = 10000
saving_growth_rate = 0.02
withdrawal = 100000
mean = 0.06
stdev = 0.1

# total years
T = years_saving + years_withdrawing

# random returns
rv = norm(loc=mean,scale=stdev)
rets = pd.DataFrame(rv.rvs((T+1)*numsims).reshape(((T+1),numsims)))

# only half year for first and last returns
rets.iloc[0] = (1+rets.iloc[0])**0.5 - 1
rets.iloc[-1] = (1+rets.iloc[-1])**0.5 - 1

# empty dataframe to store annual account balance for each simulation
# dates are 0, 0.5, 1.5, 2.5, ..., T-0.5, T
# T+1 dates in total
# initialize with initial balance
balance = pd.DataFrame(dtype=float,index=range(T+2),columns=range(numsims))
balance.iloc[0] = initial_balance

# add return and saving at 0.5, 1.5, ..., years_saving-0.5
saving = initial_saving
for i in range(years_saving) :
    balance.iloc[i+1] = balance.iloc[i]*(1+rets.iloc[i]) + saving
    saving = saving*(1+saving_growth_rate)

# add return and subtract withdrawal at years_saving+0.5, ..., T-0.5
for i in range(years_saving,T) :
    balance.iloc[i+1] = balance.iloc[i]*(1+rets.iloc[i]) - withdrawal

# add final half-year return
balance.iloc[-1] = balance.iloc[-2]*(1+rets.iloc[-1])

# we only keep the final balance
balance = balance.iloc[-1]

# compute percentiles
percentiles = balance.quantile([i/100 for i in range(1,100)])
percentiles.index.name = 'percentage'

# compute statistics and some percentiles
stats = balance.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])

In [2]:
percentiles

percentage
0.01   -4.157329e+06
0.02   -3.440308e+06
0.03   -3.002394e+06
0.04   -2.670394e+06
0.05   -2.417534e+06
            ...     
0.95    2.219620e+07
0.96    2.430027e+07
0.97    2.739257e+07
0.98    3.118044e+07
0.99    3.962000e+07
Name: 61, Length: 99, dtype: float64

In [3]:
stats

count    1.000000e+04
mean     5.874308e+06
std      8.803779e+06
min     -9.225663e+06
10%     -1.496889e+06
25%      4.256912e+05
50%      3.397389e+06
75%      8.504214e+06
90%      1.602053e+07
max      1.321481e+08
Name: 61, dtype: float64